In [0]:
from pyspark.sql import functions as F, types as T
from delta.tables import DeltaTable
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.types import *
from pyspark.sql import Window

In [0]:
def write_to_table(
    df: DataFrame,
    table_name: str,
    mode: str = "overwrite",
    merge_schema: bool = True,
    partition_by: list[str] = None,
    path: str = None,
    save_as_table: bool = True
) -> None:
    """
    Generalised Delta write helper for bronze layer.

    Parameters:
    - df (DataFrame): Spark DataFrame to write.
    - table_name (str): Name of the Delta table (used if save_as_table=True).
    - mode (str): Write mode ('overwrite', 'append', 'ignore', 'error', etc.).
    - merge_schema (bool): Whether to merge schema on write.
    - partition_by (list[str], optional): List of columns to partition by.
    - path (str, optional): Path to save the Delta table (used if save_as_table=False).
    - save_as_table (bool): If True, saves as managed table; else saves to path.

    Raises:
    - ValueError: If neither save_as_table nor path is properly specified.
    """

    df_with_ts = df.withColumn("last_updated", F.current_timestamp())

    writer = df_with_ts.write.format("delta").mode(mode)

    if merge_schema:
        writer = writer.option("mergeSchema", "true")
    elif mode == "overwrite":
        writer = writer.option("overwriteSchema", "true")

    if partition_by:
        writer = writer.partitionBy(*partition_by)

    if save_as_table:
        writer.saveAsTable(table_name)
    elif path:
        writer.save(path)
    else:
        raise ValueError("Either save_as_table must be True or a path must be provided.")

In [0]:
def merge_to_table(
    df: DataFrame,
    table_name: str,
    merge_condition: str,
    spark: SparkSession,
    partition_by: list[str] = None,
    merge_schema: bool = True
) -> None:
    """
    Performs an upsert (merge) into a Delta table.

    Parameters:
    - df (DataFrame): Incoming DataFrame to merge.
    - table_name (str): Target Delta table name.
    - merge_condition (str): SQL condition for matching rows.
    - spark (SparkSession): Active Spark session.
    - partition_by (list[str], optional): Columns to partition by on initial write.
    - merge_schema (bool): Whether to merge schema on write.

    If the table does not exist, it will be created using write_to_table.
    """
    df_with_ts = df.withColumn("last_updated", F.current_timestamp())

    if not spark.catalog.tableExists(table_name):
        write_to_table(
            df=df_with_ts,
            table_name=table_name,
            partition_by=partition_by,
            merge_schema=merge_schema
        )
    else:
        delta_table = DeltaTable.forName(spark, table_name)
        (
            delta_table.alias("target")
            .merge(
                source=df_with_ts.alias("source"),
                condition=merge_condition
            )
            .whenMatchedUpdateAll()
            .whenNotMatchedInsertAll()
            .execute()
        )


In [0]:
try:
    ENV = dbutils.widgets.get("ENV")
except Exception:
    ENV = "prod"

#ensure valid ENV and PROTOCOL
valid_envs = {"dev", "test", "prod"}

# Validate ENV
if ENV not in valid_envs:
    print(f"Invalid ENV: {ENV}. Must be one of {valid_envs}. Exiting notebook.")
    dbutils.notebook.exit("Invalid ENV")
    
silver_schema = f"fpl_silver_{ENV}"
feature_schema = f"fpl_feature_{ENV}"
gold_schema = f"fpl_gold_{ENV}"

In [0]:
position_df = spark.table(f"{silver_schema}.positions").select("position_key", "position_name", "position_name_short")
team_df = spark.table(f"{silver_schema}.teams").select("team_key", "team_season_key", "team_name_short").distinct()

player_raw_df = spark.table(f"{silver_schema}.players"
        ).withColumnRenamed(
            "team_season_key", "_team_season_key"
        ).withColumn(
            "player_name", 
            F.concat_ws(" ", F.col("first_name"), F.col("second_name"))
        ).drop("team_key")
        
        
player_df = player_raw_df.join(
            team_df, 
            on= player_raw_df["_team_season_key"] == team_df["team_season_key"], 
            how="inner"
        ).join(
            position_df, 
            on="position_key", 
            how="inner"
        )

window_spec = Window.partitionBy("player_name", "season_key").orderBy("player_surrogate_key")
player_df = player_df.withColumn(
    "is_duplicate",
    F.when(F.count("player_surrogate_key").over(window_spec) > 1, F.lit(True)).otherwise(F.lit(False))
)

player_df = player_df.withColumn(
    "player_name",
    F.when(
        F.col("is_duplicate"),
        F.concat_ws(" ", F.col("player_name"), F.col("team_name_short"))
    ).otherwise(F.col("player_name"))
)

player_df = player_df.select(
    "player_surrogate_key",
    "player_key",
    "player_season_spell_key",
    "season_key",
    "player_name",
    "player_season_key",
    "team_key",
    "team_season_key",
    "team_name_short",
    "position_name",
    "initial_value",
    "current_value",
    "effective_from",
    "effective_to"
)

merge_to_table(
    df=player_df,
    table_name=f"{gold_schema}.dim_player",
    merge_condition="target.player_surrogate_key = source.player_surrogate_key",
    spark=spark
)

In [0]:
gameweek_df = spark.table(f"{silver_schema}.gameweeks")

merge_to_table(
    df = gameweek_df,
    table_name = f"{gold_schema}.dim_gameweek",
    merge_condition = "target.gameweek_key = source.gameweek_key",
    spark = spark
)

In [0]:
team_df = spark.table(f"{silver_schema}.teams").select("team_key", "team_season_key", "team_name_short").distinct()

fixture_df = spark.table(f"{silver_schema}.fixtures").select(
    "fixture_key",
    "season_key",
    "gameweek_key",
    "home_team_key",
    "home_team_season_key",
    "away_team_key",
    "away_team_season_key",
    "home_team_score",
    "away_team_score",
    "kickoff_time",
    "gameweek"
)

fixture_df = fixture_df.join(
    team_df.withColumnRenamed("team_season_key", "home_team_season_key"
        ).withColumnRenamed("team_name_short", "home_team_name"),
    on="home_team_season_key",
    how="left"
).join(
    team_df.withColumnRenamed("team_key", "away_team_season_key"
        ).withColumnRenamed("team_name_short", "away_team_name"),
    on="away_team_season_key",
    how="left"
)

merge_to_table(
    df = fixture_df,
    table_name = f"{gold_schema}.dim_fixture",
    merge_condition = "target.fixture_key = source.fixture_key",
    spark = spark
)

In [0]:
players = spark.table(f"{silver_schema}.players").select(
    "player_surrogate_key",
    "player_key",
    "effective_from",
    "effective_to"
)

player_stats = spark.table(f"{silver_schema}.gameweek_stats").select(
    "player_key",
    "season_key",
    "fixture_key",
    "gameweek_key",
    "team_key",
    "team_season_key",
    "position_key",
    "opponent_team_key",
    "opponent_team_season_key",
    "minutes",
    "assists",
    "goals_scored",
    "goals_conceded",
    "clean_sheets",
    "own_goals",
    "bps",
    "clearances_blocks_interceptions",
    "recoveries",
    "creativity",
    "defensive_contribution",
    "tackles",
    "saves",
    "yellow_cards",
    "red_cards",
    "expected_assists",
    "expected_goals",
    "expected_goal_involvements",
    "expected_goals_conceded",
    "penalties_saved",
    "penalties_missed",
    "ict_index",
    "influence",
    "value",
    "was_home"
)

player_stats = player_stats.join(
    players,
    (player_stats.player_key == players.player_key) &
    (player_stats.gameweek_key >= players.effective_from) &
    (player_stats.gameweek_key <= players.effective_to),
    "left"
).drop(
    "effective_from",
    "effective_to",
    players.player_key
).withColumn(
    "player_gw_stat_key",
    F.concat(F.col("player_surrogate_key"), F.col("fixture_key")).cast("long")
)

merge_to_table(
    df = player_stats,
    table_name = f"{gold_schema}.fact_player_gameweek_stats",
    merge_condition = "target.player_gw_stat_key = source.player_gw_stat_key",
    spark = spark
)

In [0]:
players = spark.table(f"{silver_schema}.players").select(
    "player_surrogate_key",
    "player_key",
    "effective_from",
    "effective_to"
)

player_points = spark.table(f"{silver_schema}.gameweek_stats").select(
    "player_surrogate_key",
    "season_key",
    "fixture_key",
    "gameweek_key",
    "team_key",
    "team_season_key",
    "position_key",
    "opponent_team_key",
    "opponent_team_season_key",
    "minutes_points",
    "assist_points",
    "goal_points",
    "clean_sheet_points",
    "defensive_contribution_points",
    "goals_conceded_points",
    "own_goal_points",
    "penalty_miss_points",
    "penalty_saves_points",
    "tackles",
    "saves_points",
    "yellow_card_points",
    "red_card_points",
    F.col("bonus").alias("bonus_points")
)

player_points = player_points.join(
    players,
    (player_points.player_surrogate_key == players.player_surrogate_key),
    "left"
).drop(
    "effective_from",
    "effective_to",
    players.player_surrogate_key
).withColumn(
    "player_gw_stat_key",
    F.concat(F.col("player_surrogate_key"), F.col("fixture_key")).cast("long")
).withColumn(
    "total_points",
    F.coalesce(F.col("minutes_points"), F.lit(0)) +
    F.coalesce(F.col("assist_points"), F.lit(0)) +
    F.coalesce(F.col("goal_points"), F.lit(0)) +
    F.coalesce(F.col("clean_sheet_points"), F.lit(0)) +
    F.coalesce(F.col("defensive_contribution_points"), F.lit(0)) +
    F.coalesce(F.col("goals_conceded_points"), F.lit(0)) +
    F.coalesce(F.col("own_goal_points"), F.lit(0)) +
    F.coalesce(F.col("penalty_miss_points"), F.lit(0)) +
    F.coalesce(F.col("penalty_saves_points"), F.lit(0)) +
    F.coalesce(F.col("saves_points"), F.lit(0)) +
    F.coalesce(F.col("yellow_card_points"), F.lit(0)) +
    F.coalesce(F.col("red_card_points"), F.lit(0)) +
    F.coalesce(F.col("bonus_points"), F.lit(0))
)

merge_to_table(
    df = player_points,
    table_name = f"{gold_schema}.fact_player_fpl_points",
    merge_condition = "target.player_gw_stat_key = source.player_gw_stat_key",
    spark = spark
)

In [0]:
player_stats = spark.table(f"{silver_schema}.gameweek_stats").select(
    "team_key",
    "team_season_key",
    "fixture_key",
    "gameweek_key",
    "season_key",
    "minutes",
    "assists",
    "goals_scored",
    "goals_conceded",
    "clean_sheets",
    "own_goals",
    "bps",
    "clearances_blocks_interceptions",
    "recoveries",
    "creativity",
    "defensive_contribution",
    "tackles",
    "saves",
    "yellow_cards",
    "red_cards",
    "expected_assists",
    "expected_goals",
    "expected_goal_involvements",
    "expected_goals_conceded",
    "penalties_saved",
    "penalties_missed",
    "ict_index",
    "influence",
    "value",
    "opponent_team_key",
    "opponent_team_season_key"
)

agg_exprs = [
    F.sum("minutes").alias("minutes"),
    F.sum("assists").alias("assists"),
    F.sum("goals_scored").alias("goals_scored_raw"),
    F.max("goals_conceded").alias("goals_conceded"),
    F.max("clean_sheets").alias("clean_sheets"),
    F.sum("own_goals").alias("own_goals"),
    F.sum("bps").alias("bps"),
    F.sum("clearances_blocks_interceptions").alias("clearances_blocks_interceptions"),
    F.sum("recoveries").alias("recoveries"),
    F.round(F.sum("creativity"), 3).alias("creativity"),
    F.sum("defensive_contribution").alias("defensive_contribution"),
    F.sum("tackles").alias("tackles"),
    F.sum("saves").alias("saves"),
    F.sum("yellow_cards").alias("yellow_cards"),
    F.sum("red_cards").alias("red_cards"),
    F.round(F.sum("expected_assists"), 3).alias("expected_assists"),
    F.round(F.sum("expected_goals"), 3).alias("expected_goals"),
    F.round(F.sum("expected_goal_involvements"), 3).alias("expected_goal_involvements"),
    F.round(F.sum("expected_goals_conceded"), 3).alias("expected_goals_conceded"),
    F.sum("penalties_saved").alias("penalties_saved"),
    F.sum("penalties_missed").alias("penalties_missed")
]

team_gw_stats = player_stats.groupBy(
    "team_season_key", "fixture_key", "gameweek_key", "season_key"
).agg(*agg_exprs)

# Calculate sum of own_goals from players with opponent_team_key for the same fixture
own_goals_against_df = player_stats.groupBy(
    "opponent_team_season_key", "fixture_key", "gameweek_key", "season_key"
).agg(
    F.sum("own_goals").alias("own_goals_against")
).withColumnRenamed("opponent_team_season_key", "team_season_key")

team_gw_stats = team_gw_stats.join(
    own_goals_against_df,
    on=["team_season_key", "fixture_key", "gameweek_key", "season_key"],
    how="left"
).withColumn(
    "goals_scored",
    F.col("goals_scored_raw") + F.coalesce(F.col("own_goals_against"), F.lit(0))
).drop("goals_scored_raw", "own_goals_against")

team_gw_stats = team_gw_stats.withColumn(
    "team_gw_stat_key",
    F.concat(F.col("team_season_key"), F.col("fixture_key")).cast("long")
)

merge_to_table(
    df = team_gw_stats,
    table_name = f"{gold_schema}.fact_team_gameweek_stats",
    merge_condition = "target.team_gw_stat_key = source.team_gw_stat_key",
    spark = spark
)

In [0]:
#fact table as well as dim to expand for future reference - can add in match stats, scorers, attendance etc.
team_df = spark.table(f"{silver_schema}.teams").select("team_season_key", "team_name_short").distinct()

fixture_df = spark.table(f"{silver_schema}.fixtures").select(
    "fixture_key",
    "season_key",
    "gameweek_key",
    "home_team_key",
    "home_team_season_key",
    "away_team_key",
    "away_team_season_key",
    "home_team_score",
    "away_team_score",
    "kickoff_time",
    "gameweek"
)

fixture_df = fixture_df.join(
    team_df.withColumnRenamed("team_season_key", "home_team_season_key"
        ).withColumnRenamed("team_name_short", "home_team_name"),
    on="home_team_season_key",
    how="left"
).join(
    team_df.withColumnRenamed("team_season_key", "away_team_season_key"
        ).withColumnRenamed("team_name_short", "away_team_name"),
    on="away_team_season_key",
    how="left"
)

merge_to_table(
    df = fixture_df,
    table_name = f"{gold_schema}.fact_fixture",
    merge_condition = "target.fixture_key = source.fixture_key",
    spark = spark
)

In [0]:
players = spark.table(f"{silver_schema}.players").select(
    "player_surrogate_key",
    "team_season_key",
    "position_key",
    "effective_from",
    "effective_to"
)

player_predicted_points = spark.table(f"{feature_schema}.player_points_inference").select(
    "player_surrogate_key",
    "fixture_key",
    "predicted_total_points"
)

fixture_gw = spark.table(f"{silver_schema}.fixtures").select(
    "fixture_key",
    "gameweek_key",
    "season_key",
    "home_team_season_key",
    "away_team_season_key"
)

player_predicted_points = player_predicted_points.join(
    fixture_gw,
    on="fixture_key",
    how="left"
)

player_points = player_predicted_points.join(
    players,
    (player_predicted_points.player_surrogate_key == players.player_surrogate_key) &
    (player_predicted_points.gameweek_key >= (players.effective_from)) &
    (player_predicted_points.gameweek_key <= (players.effective_to + 6)),
    "left"
).drop(
    "effective_from",
    "effective_to",
    players.player_surrogate_key
).withColumn(
    "player_gw_stat_key",
    F.concat(F.col("player_surrogate_key"), F.col("fixture_key")).cast("long")
).withColumn(
    "opponent_team_season_key",
    F.when(
        F.col("team_season_key") == F.col("home_team_season_key"),
        F.col("away_team_season_key")
    ).otherwise(F.col("home_team_season_key"))
).withColumn(
    "is_home",
    F.col("team_season_key") == F.col("home_team_season_key")
).drop(
    "home_team_season_key",
    "away_team_season_key"
)

fpl_points = spark.table(f"{gold_schema}.fact_player_fpl_points").select(
    F.col("player_gw_stat_key").alias("_player_gw_stat_key"),
    "total_points"
)

player_predicted_actual_points = player_points.join(
    fpl_points,
    player_points.player_gw_stat_key == fpl_points._player_gw_stat_key,
    "left"
).drop("_player_gw_stat_key"
).withColumn(
    "prediction_variance",
    F.col("total_points") - F.col("predicted_total_points")
).withColumn(
    "prediction_variance_abs",
    F.abs(F.col("total_points") - F.col("predicted_total_points"))
)

merge_to_table(
    df = player_predicted_actual_points,
    table_name = f"{gold_schema}.fact_fpl_point_prediction_performance",
    merge_condition = "target.player_gw_stat_key = source.player_gw_stat_key",
    spark = spark
)

In [0]:
team_features_df = spark.table(f"{feature_schema}.team_features").withColumn(
    "team_season_gw_key",
    F.concat(F.col("team_season_key"), F.col("fixture_key")).cast("long")
)

merge_to_table(
    df = team_features_df,
    table_name = f"{gold_schema}.fact_team_features",
    merge_condition = "target.team_season_gw_key = source.team_season_gw_key",
    spark = spark
)

In [0]:
player_features_df = spark.table(f"{feature_schema}.player_features").withColumn(
    "player_gw_stat_key",
    F.concat(F.col("player_surrogate_key"), F.col("fixture_key")).cast("long")
)

merge_to_table(
    df = player_features_df,
    table_name = f"{gold_schema}.fact_player_features",
    merge_condition = "target.player_gw_stat_key = source.player_gw_stat_key",
    spark = spark
)

In [0]:
# fpl_prediction_df = spark.table(f"{gold_schema}.fact_fpl_point_prediction_performance").select(
#     "player_surrogate_key",
#     "position_key",
#     "fixture_key",
#     "gameweek_key",
#     "season_key",
#     "team_season_key",
#     "opponent_team_season_key",
#     "is_home",
#     "predicted_total_points"
# )

# fpl_player_price = spark.table(f"{silver_schema}.players").select(
#     "player_surrogate_key",
#     "current_value"
# ) #no history of previous value

# fpl_df = fpl_prediction_df.join(
#     fpl_player_price,
#     on="player_surrogate_key",
#     how="left"
# )

# # Position constraints
# position_limits = {1: 2, 2: 5, 3: 5, 4: 3}
# max_team_players = 3
# max_total_value = 1000

# # Add totw_gameweek_key
# fpl_df = fpl_df.withColumn("totw_gameweek_key", F.col("gameweek_key"))

# # Window for ranking by predicted points within each gameweek, position, and team
# window_gw_pos = Window.partitionBy("gameweek_key", "position_key").orderBy(F.col("predicted_total_points").desc())
# window_gw_team = Window.partitionBy("gameweek_key", "team_season_key").orderBy(F.col("predicted_total_points").desc())

# # Rank within position and team constraints
# fpl_df = fpl_df.withColumn("pos_rank", F.row_number().over(window_gw_pos))
# fpl_df = fpl_df.withColumn("team_rank", F.row_number().over(window_gw_team))

# # Filter by position and team constraints
# fpl_df_filtered = fpl_df.filter(
#     (F.col("pos_rank") <= F.create_map(
#         *[item for kv in position_limits.items() for item in (F.lit(kv[0]), F.lit(kv[1]))]
#     )[F.col("position_key")]) &
#     (F.col("team_rank") <= max_team_players)
# )

# display(fpl_df_filtered)

# def select_totw(pdf):
#     import pandas as pd
#     from itertools import combinations, product

#     pos_groups = {k: pdf[pdf.position_key == k] for k in position_limits}
#     pos_combos = [list(combinations(pos_groups[k].index, position_limits[k])) for k in position_limits]
#     all_combos = product(*pos_combos)
#     best_combo = None
#     best_points = -float("inf")
#     for combo in all_combos:
#         idxs = [i for group in combo for i in group]
#         team = pdf.loc[list(idxs)]
#         if team.groupby("team_season_key").size().max() > max_team_players:
#             continue
#         if team["current_value"].sum() > max_total_value:
#             continue
#         points = team["predicted_total_points"].sum()
#         if points > best_points:
#             best_points = points
#             best_combo = team
#     if best_combo is not None:
#         return best_combo
#     return pd.DataFrame(columns=pdf.columns)

# totw_df = fpl_df_filtered.groupBy("gameweek_key").applyInPandas(select_totw, schema=fpl_df_filtered.schema)

# display(totw_df)

player_surrogate_key,position_key,fixture_key,gameweek_key,season_key,team_season_key,opponent_team_season_key,is_home,predicted_total_points,current_value,totw_gameweek_key,pos_rank,team_rank
14511571,1,202526023,20252603,202526,2025268,20252654,true,1.4464412,43,20252603,1,1
76131100,1,202526022,20252603,202526,20252636,20252643,true,1.4464412,45,20252603,2,1
87172650,1,202526039,20252604,202526,2025264,20252639,true,4.881626,51,20252604,2,2
557392893,2,202526040,20252604,202526,2025266,20252621,false,6.6626306,50,20252604,1,1
431006638,3,202526040,20252604,202526,2025266,20252621,false,5.8588614,65,20252604,4,2
1444705699,2,202526040,20252604,202526,2025266,20252621,false,5.804077,46,20252604,4,3
210211534,2,202526033,20252604,202526,2025268,20252694,false,6.398787,55,20252604,2,1
1623854474,3,202526033,20252604,202526,2025268,20252694,false,6.296788,65,20252604,3,2
1965861697,3,202526036,20252604,202526,20252611,2025267,true,6.899603,49,20252604,1,1
496196026,3,202526034,20252604,202526,20252614,20252690,false,5.637011,140,20252604,5,1


player_surrogate_key,position_key,fixture_key,gameweek_key,season_key,team_season_key,opponent_team_season_key,is_home,predicted_total_points,current_value,totw_gameweek_key,pos_rank,team_rank
87172650,1,202526100,20252610,202526,2025264,20252621,false,5.2594748,51,20252610,2,2
311520959,1,202526095,20252610,202526,2025267,20252614,false,5.513585,50,20252610,1,1
930937896,2,202526092,20252610,202526,2025263,20252690,false,6.6917295,68,20252610,2,1
1784880161,2,202526092,20252610,202526,2025263,20252690,false,5.6109333,63,20252610,5,3
1444705699,2,202526099,20252610,202526,2025266,2025268,true,8.617489,46,20252610,1,1
476142121,2,202526091,20252610,202526,20252636,2025262,true,6.1808496,45,20252610,4,2
1412411279,2,202526094,20252610,202526,20252654,20252639,true,6.272672,46,20252610,3,2
597952188,3,202526097,20252610,202526,2025261,20252617,false,7.611867,55,20252610,1,1
117864615,3,202526092,20252610,202526,2025263,20252690,false,5.8298974,102,20252610,3,2
1996760399,3,202526098,20252610,202526,20252611,20252656,false,5.7757134,62,20252610,4,1
